In [209]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.manifold import MDS
from sklearn.preprocessing import Imputer,StandardScaler,MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
import seaborn as sns
%matplotlib inline

In [128]:
X = pd.read_csv('S&P500_2017.csv')
X = X.drop(['Date'],axis=1)

In [129]:
X.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
0,46.074703,45.816051,170.237915,113.847588,59.784031,80.855621,37.982937,114.158501,103.480003,70.577911,...,36.900692,57.513260,86.670731,58.296070,26.833357,48.875042,61.848686,102.513084,42.586197,53.118965
1,46.679256,46.211868,171.634933,113.720161,60.626999,82.862099,38.284462,114.432961,104.139999,70.431908,...,37.027840,57.094593,85.717155,58.773418,27.845938,49.603493,62.073734,103.455551,43.197666,53.634392
2,46.124260,45.410336,171.515182,114.298462,61.086800,81.902901,38.615170,112.717552,105.910004,69.419624,...,36.607288,56.403305,84.439362,58.007671,27.729099,49.150673,62.279217,104.120262,42.497429,53.455975
3,47.561314,45.726990,169.269974,115.572708,61.105961,82.999107,39.665661,114.001671,108.300003,69.692154,...,36.568165,57.493786,84.391678,57.967899,27.300703,48.806133,63.032623,104.130180,42.773582,53.624477
4,47.709969,46.587898,169.170197,116.631294,61.508286,83.664680,39.626755,112.727356,108.570000,70.023094,...,35.952015,57.503525,82.999451,58.146900,27.144920,48.599411,63.208748,106.154053,42.310051,53.475800


In [181]:
Times = [(t1,t1+np.random.choice(range(1,30))) for t1 in [np.random.choice(range(220))for _ in range(1000)] ]

In [137]:
M=X.fillna(X.mean())
M = M.pct_change()
Z = pd.DataFrame(columns=M.columns)
Z = Z.T
Z['Mean'] = M.mean()
Z['Std'] = M.std()
Z['Skew'] = M.skew()
Z['Kurt'] = M.kurt()

In [194]:
Y = pd.DataFrame(columns=['Mean','Std','Skew','Kurt','I','F','D'],index=range(1000))
for i in range(1000):
    r = np.random.choice(503)
    d=  Times[i][1]-np.argmax(X.T.iloc[r][Times[i][0]:Times[i][1]])
    Y.iloc[i]=list(Z.iloc[r])+[Times[i][0],Times[i][1],d]

In [195]:
Y = Y.dropna()
Y.describe()

,Mean,Std,Skew,Kurt,I,F,D
count,999.000000,999.000000,999.000000,999.000000,999,999,999
unique,437.000000,437.000000,437.000000,437.000000,219,234,28
top,0.001291,0.008793,0.449715,3.276704,185,108,1
freq,7.000000,7.000000,7.000000,7.000000,11,12,237


In [212]:
W = Y[['Mean','Std','Skew','Kurt','I','F']]
y = Y['D']

In [201]:
imputer = Imputer(strategy='mean')
scaler = MinMaxScaler()
pca = PCA()
pipe = make_pipeline(imputer,pca,scaler)
pipe.fit(W)

Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1)))])

In [204]:
Xp=pd.DataFrame(pipe.transform(W),columns=['p%d'%x for x in range(1,7)])
param_grid= dict(hidden_layer_sizes=[(i,) for i in range(5,21)],
activation = ['identity', 'logistic', 'tanh', 'relu'],
solver = ['lbfgs', 'sgd', 'adam'],
learning_rate = ['constant', 'invscaling', 'adaptive'],
max_iter = [10000])

In [213]:
%%time 
model = MLPClassifier()
grid = GridSearchCV(cv=4,n_jobs=-1,scoring='roc_auc',
                    estimator=model,
                    param_grid=param_grid)

grid.fit(Xp,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/aleespa/anaconda2/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/home/aleespa/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

......................................

In [211]:
y

,D
0,8
1,2
2,4
3,2
4,1
5,13
6,7
7,2
8,10
9,20
